In [76]:
%pip install "git+https://github.com/ScierKnave/TorchMPS.git"
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
%pip install torchmetrics
from torchmetrics.classification import MulticlassAccuracy
from torchmps import MPS
import math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ScierKnave/TorchMPS.git to /tmp/pip-req-build-___r27tp
  Running command git clone --filter=blob:none --quiet https://github.com/ScierKnave/TorchMPS.git /tmp/pip-req-build-___r27tp
  Resolved https://github.com/ScierKnave/TorchMPS.git to commit f716a08e15d0af50dbfdfc435ab9604e82562ea3
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Hyperparameters

In [77]:
# FC to 2-MPS

# Hardware hyperparameters
chosen_device = torch.device('cuda' 
if torch.cuda.is_available() else 'cpu')

# Data hyperparameters
nb_train_HP = 2000
nb_test_HP = 500
batch_sz_HP = 150
batch_sz_HP = min(batch_sz_HP, nb_train_HP)
nb_classes_HP = 10

# Teacher hyperparameters
teacher_hidden_size_HP = 10

# Student hyperparameters
# MPS parameters
bond_dim_HP = 20
adaptive_mode_HP = False
periodic_bc_HP = False
use_cust_feature_map_HP = False

# Training parameters
nepochs_student_HP = 25 
student_lr_HP = 1e-4
student_reg_HP = 0.00
student_loss_HP = nn.CrossEntropyLoss()



# Premilinaries: Importing the data and utils subroutines

In [78]:
# Import the mnist train dataset
train_set = torchvision.datasets.MNIST(
    root = './datasets', train = True,   
    transform = transforms.ToTensor(),  download = True )

# Create a training batch iterator
train_subset = torch.utils.data.SubsetRandomSampler(range(nb_train_HP))
train_iterator = torch.utils.data.DataLoader(
    dataset = train_set, 
    sampler = train_subset, batch_size=batch_sz_HP
    )

# Import the mnist test set
test_set = torchvision.datasets.MNIST(
    root = './datasets',
    train = False, transform = transforms.ToTensor(),  download = True
    )
# Create a testing batch iterator
test_subset = torch.utils.data.SubsetRandomSampler(range(nb_test_HP))
test_iterator = torch.utils.data.DataLoader(
    dataset = test_set, 
    sampler = test_subset, batch_size = batch_sz_HP
    )

In [79]:
# Returns the validation set classification accuracy
# of the given input model (this is a higher order function)
def get_acc(model, iterator):
    # Get the validation set classification accuracy
    total_good_classifications = 0
    acc_metric = MulticlassAccuracy(num_classes=nb_classes_HP).to(chosen_device)
    for (x_mb, y_mb) in iterator:
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        y_mb = y_mb.to(chosen_device)
        # Add the number of datapoints we classified right to the total
        batch_size = x_mb.size()[0]
        y_hat = model(x_mb)
        batch_good_classifications = batch_size * acc_metric(y_hat, y_mb)
        total_good_classifications += batch_good_classifications
    return total_good_classifications / nb_test_HP # divide by total size

# Training the student model

In [80]:
class feature_map_HP(nn.Module):
    def __init__(self):
        super(feature_map_HP, self).__init__()
    def forward(self, x):
        return torch.tensor([1, x], device=chosen_device, requires_grad=False)


class Student(nn.Module):
    ''' 
    Our student model is a composition of two MPS.
    '''
    def __init__(self):
        super(Student, self).__init__()
        # Initialize the MPS modules
        self.mps1 = MPS(
            input_dim = 28 ** 2,
            feature_dim = 2,
            output_dim = teacher_hidden_size_HP,
            bond_dim = bond_dim_HP
          )

        self.mps2 = MPS(
          input_dim = teacher_hidden_size_HP, 
          feature_dim = 2,
          output_dim = nb_classes_HP,
          bond_dim = bond_dim_HP
        )
        
        self.feature_map_HP = feature_map_HP().to(chosen_device)
        if use_cust_feature_map_HP: 
          self.mps1.register_feature_map(self.feature_map_HP)
          self.mps2.register_feature_map(self.feature_map_HP)


    def forward(self, x):
        y = self.mps1(x)
        y = self.mps2(y)
        return y

student = Student().to(chosen_device)


# Create the optimizers for the training
mps1_optimizer = torch.optim.Adam(
    student.mps1.parameters(), lr = student_lr_HP, weight_decay = student_reg_HP
)

mps2_optimizer = torch.optim.Adam(
    student.mps2.parameters(), lr = student_lr_HP, weight_decay = student_reg_HP
)

student_optimizer = torch.optim.Adam(
    student.parameters(), lr = student_lr_HP, weight_decay = student_reg_HP
)

# Used on the inputs before the loss function
LogSoftmax = nn.LogSoftmax(dim=1)

# Create an array to store the val loss
# of the student at each epoch
stud_test_loss = []
stud_train_loss = []

softmax = nn.Softmax(dim=1)
# Training loop 
for epoch in range(4 * nepochs_student_HP):
    for (x_mb, y_mb) in train_iterator:
        # Flatten the MNIST images, which come in matrix form
        x_mb = x_mb.reshape(-1, 784).to(chosen_device)
        y_mb = y_mb.to(chosen_device)

        # Train mps1
        if (epoch < nepochs_student_HP):
          mps1_logits = student.mps1(x_mb) 
          # Backpropagation
          loss = nn.CrossEntropyLoss()(mps1_logits, y_mb)
          loss.backward()
          mps1_optimizer.step()
          mps1_optimizer.zero_grad()
          

        # Train entire student
        else:
          student_logits = student(x_mb) 
          # Backpropagation
          loss = student_loss_HP(student_logits, y_mb)
          loss.backward()
          mps2_optimizer.step()
          mps2_optimizer.zero_grad()


    #stud_train_loss.append( round(get_acc(student, train_iterator).item(), 3) )
    stud_test_loss.append( round(get_acc(student, test_iterator).item(), 5) )
    print(stud_test_loss)
    print(stud_train_loss)
    print(loss)


print("Epochs: ", np.arange(1, nepochs_student_HP+1).tolist())
print("Train loss: ", stud_train_loss)
print("Test loss: ", stud_test_loss)


[0.1]
[]
tensor(2.2836, device='cuda:0', grad_fn=<NllLossBackward0>)
[0.1, 0.1]
[]
tensor(2.3025, device='cuda:0', grad_fn=<NllLossBackward0>)
[0.1, 0.1, 0.1]
[]
tensor(2.3024, device='cuda:0', grad_fn=<NllLossBackward0>)
[0.1, 0.1, 0.1, 0.1]
[]
tensor(2.3020, device='cuda:0', grad_fn=<NllLossBackward0>)
[0.1, 0.1, 0.1, 0.1, 0.1]
[]
tensor(2.3002, device='cuda:0', grad_fn=<NllLossBackward0>)
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[]
tensor(2.2354, device='cuda:0', grad_fn=<NllLossBackward0>)
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[]
tensor(1.5782, device='cuda:0', grad_fn=<NllLossBackward0>)
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[]
tensor(1.0559, device='cuda:0', grad_fn=<NllLossBackward0>)
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[]
tensor(1.0066, device='cuda:0', grad_fn=<NllLossBackward0>)
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[]
tensor(1.0218, device='cuda:0', grad_fn=<NllLossBackward0>)
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[]
tensor(0.7709, device='cud